In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlp-model10-1200/MLP_model (11).h5
/kaggle/input/mlp-model11-reduced-test-size/MLP_model (12).h5
/kaggle/input/mlp-model12-without-test/MLP_model (13).h5
/kaggle/input/mlp-model7/MLP_model (8).h5
/kaggle/input/mlp-model2/MLP_model (3).h5
/kaggle/input/mlp-model5/MLP_model (6).h5
/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/mlp-model8-771/MLP_model (9).h5
/kaggle/input/mlp-model9-1000/MLP_model (10).h5
/kaggle/input/mlp-mo

In [2]:
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
print(train_terms.shape)

(5363863, 3)


In [3]:
import tensorflow as tf
# 저장된 모델 불러오기
model = tf.keras.models.load_model('/kaggle/input/mlp-model11-reduced-test-size/MLP_model (12).h5')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
test_embeddings = np.load('/kaggle/input/t5embeds/test_embeds.npy')
test_id = pd.DataFrame(np.load('/kaggle/input/t5embeds/test_ids.npy'))

labels = train_terms['term'].value_counts().index[:1200].tolist()

# Convert test_embeddings to dataframe
column_num = test_embeddings.shape[1]
test_df = pd.DataFrame(test_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
print(test_df.shape)

(141865, 1024)


In [5]:
predictions =  model.predict(test_df)
predictions

4434/4434 [==============================] - 10s 2ms/step


array([[6.9237471e-01, 7.0257032e-01, 6.8430156e-01, ..., 1.1715016e-03,
        1.3470497e-03, 3.5478938e-03],
       [7.4585718e-01, 7.2594625e-01, 7.3881686e-01, ..., 2.3844764e-03,
        1.8532773e-03, 5.5788197e-03],
       [7.1934670e-01, 7.3416317e-01, 7.1147412e-01, ..., 2.0994353e-03,
        8.9048123e-04, 7.2243800e-03],
       ...,
       [3.1443793e-01, 6.5855783e-01, 2.9192594e-01, ..., 4.2242944e-05,
        1.2531194e-04, 1.1926375e-04],
       [3.4951240e-01, 6.5999323e-01, 3.2777390e-01, ..., 1.0966595e-04,
        1.6375643e-04, 1.5726026e-04],
       [4.5942765e-01, 6.6005892e-01, 4.5971203e-01, ..., 6.9984933e-05,
        6.2273454e-04, 3.9885932e-04]], dtype=float32)

In [6]:
predictions = pd.DataFrame(predictions)
test_id.columns = ['Protein_ID']
pred_df = pd.concat([test_id, predictions], axis=1)
pred_df.set_index('Protein_ID', inplace=True)
pred_df.columns = labels
pred_df

,GO:0005575,GO:0008150,GO:0110165,GO:0003674,GO:0005622,GO:0009987,GO:0043226,GO:0043229,GO:0005488,GO:0043227,...,GO:0008652,GO:0033646,GO:0005930,GO:0055074,GO:0048515,GO:0015081,GO:0072522,GO:1905368,GO:0048704,GO:0009582
Protein_ID,,,,,,,,,,,,,,,,,,,,,
Q9CQV8,0.692375,0.702570,0.684302,0.659737,0.607045,0.461994,0.478053,0.455063,0.656592,0.413393,...,1.560566e-04,0.001910,0.004042,0.003962,0.007467,9.489262e-05,1.620843e-04,0.001172,0.001347,0.003548
P62259,0.745857,0.725946,0.738817,0.693453,0.658088,0.535945,0.551942,0.526895,0.684225,0.473731,...,4.582043e-04,0.002722,0.007646,0.009599,0.010273,3.295335e-04,6.134348e-04,0.002384,0.001853,0.005579
P68510,0.719347,0.734163,0.711474,0.670764,0.638079,0.544530,0.506015,0.485718,0.647047,0.411370,...,3.522530e-04,0.002629,0.005583,0.004642,0.008378,1.109522e-04,3.917555e-04,0.002099,0.000890,0.007224
P61982,0.725249,0.744441,0.716388,0.675466,0.642500,0.551664,0.507917,0.487224,0.656580,0.407568,...,3.126587e-04,0.002733,0.007339,0.004675,0.012520,7.240505e-05,2.272448e-04,0.002267,0.001475,0.006642
O70456,0.668797,0.703553,0.659741,0.644751,0.604590,0.480043,0.466734,0.449062,0.622589,0.385083,...,3.187407e-04,0.002879,0.004732,0.003259,0.007226,9.926919e-05,3.266188e-04,0.001910,0.001061,0.005658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P08380,0.502896,0.481531,0.468847,0.351530,0.016422,0.033306,0.005889,0.004185,0.217432,0.005309,...,4.928594e-09,0.002304,0.000006,0.000009,0.000118,1.562821e-09,1.858735e-07,0.000010,0.000001,0.000022
C0HK72,0.348229,0.601645,0.324898,0.443901,0.016219,0.071840,0.010584,0.007053,0.250859,0.007997,...,5.606453e-09,0.000167,0.000010,0.000006,0.002203,1.844353e-09,1.352894e-08,0.000068,0.000073,0.000089
C0HK73,0.314438,0.658558,0.291926,0.388326,0.012364,0.087367,0.009310,0.006032,0.217101,0.006873,...,4.311787e-09,0.000083,0.000008,0.000008,0.003353,3.668878e-09,1.302257e-08,0.000042,0.000125,0.000119


In [7]:
temp_list = ['Label1', 'Label2', 'Label3']
df_submission = pred_df.stack().reset_index()
df_submission.columns = ['Protein Id','GO Term Id','Prediction']

In [8]:
df_submission

,Protein Id,GO Term Id,Prediction
0,Q9CQV8,GO:0005575,6.923747e-01
1,Q9CQV8,GO:0008150,7.025703e-01
2,Q9CQV8,GO:0110165,6.843016e-01
3,Q9CQV8,GO:0003674,6.597372e-01
4,Q9CQV8,GO:0005622,6.070454e-01
...,...,...,...
170237995,A0A3G2FQK2,GO:0015081,4.052344e-08
170237996,A0A3G2FQK2,GO:0072522,2.119885e-07
170237997,A0A3G2FQK2,GO:1905368,6.998493e-05
170237998,A0A3G2FQK2,GO:0048704,6.227345e-04


In [9]:
df_submission.to_csv("submission.tsv",header=False, index=False, sep="\t")